<a href="https://colab.research.google.com/github/andrejarenkow/Jornada-Academica-Bioinfo-2023/blob/main/DBF_Dengue.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install dbfread

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install --upgrade gspread

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 2.6 MB/s eta 0:00:00
  Attempting uninstall: gspread
    Found existing installation: gspread 3.4.2
    Uninstalling gspread-3.4.2:
      Successfully uninstalled gspread-3.4.2


In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [ ]:
import sys
import csv
from dbfread import DBF
import pandas as pd
import re
import os
import zipfile
import folium
import json
import requests
import matplotlib.pyplot as plt
import matplotlib.patheffects as pe

# Criando dataframes

## Municípios

In [ ]:
municipios_rs = pd.read_csv('https://raw.githubusercontent.com/decao88/csv/master/Munic%C3%ADpios%20RS%20IBGE6%20Popula%C3%A7%C3%A3o%20CRS%20Regional%20-%20P%C3%A1gina1.csv')
municipios_14 = municipios_rs[municipios_rs['CRS']==14]
#municipios_14

In [ ]:
municipios = pd.read_csv('https://raw.githubusercontent.com/decao88/csv/master/populacao%20ibge%206%20municipio%20br.csv', sep=',')
municipios['ID_MN_RESI'] = municipios['IBGE6']
#municipios.head()

## Função de limpeza do DBF da dengue

In [ ]:
def limpeza_dados(arquivo_dbf):

  dbf = DBF(arquivo_dbf, encoding='latin-1')
  dados = pd.DataFrame(iter(dbf))
  dados = dados.filter(['NM_PACIENT', 'DT_NOTIFIC','SG_UF_NOT', 'ID_MUNICIP', 'DT_SIN_PRI', 'SEM_PRI', 'NU_IDADE_N', 'CS_SEXO', 'SG_UF', 'ID_MN_RESI',
                        'CLASSI_FIN','CRITERIO','NM_LOGRADO', 'NU_NUMERO','NM_COMPLEM', 'NM_BAIRRO', 'FEBRE','MIALGIA','CEFALEIA','EXANTEMA','VOMITO',
                        'NAUSEA','DOR_COSTAS','CONJUNTVIT','ARTRITE',	'ARTRALGIA','PETEQUIA_N','LEUCOPENIA','LACO','DOR_RETRO','HOSPITALIZ','DS_OBS',
                        'DT_DIGITA', 'TPAUTOCTO', 'EVOLUCAO', 'NU_NOTIFIC'])
  dados['DT_NOTIFIC'] = pd.to_datetime(dados['DT_NOTIFIC'])
  dados['DT_SIN_PRI'] = pd.to_datetime(dados['DT_SIN_PRI'])
  dados['DT_DIGITA'] = pd.to_datetime(dados['DT_DIGITA'])
  dados['ID_MN_RESI']=pd.to_numeric(dados['ID_MN_RESI'])

  colunas_mudar = ['SG_UF_NOT', 'ID_MUNICIP', 'SEM_PRI', 'SG_UF', 'ID_MN_RESI','CLASSI_FIN',
                        'FEBRE','MIALGIA','CEFALEIA','EXANTEMA','VOMITO','NAUSEA','DOR_COSTAS','CONJUNTVIT','ARTRITE',	'ARTRALGIA',
                        'PETEQUIA_N','LEUCOPENIA','LACO','DOR_RETRO','HOSPITALIZ','CLASSI_FIN','CRITERIO']

# colocando idades
  idades = []
  for i in dados['NU_IDADE_N']:
    if i > 4000:
      idade = i - 4000

    elif 4000 > i:
      idade = 0

    idades.append(idade)

  dados['idade'] = idades

# colocando finalização
  classif_final_s = []
  for i in dados['CLASSI_FIN']:
    if i == '5':
      classificacao = 'Descartado'

    elif i == '10':
      classificacao = 'Dengue'

    elif i == '11':
      classificacao = 'Dengue com sinais de alarme'

    elif i == '12':
      classificacao = 'Dengue grave'

    elif i == '13':
      classificacao = 'Chikungunya'

    elif i == '8':
      classificacao = 'Fechado pelo sistema'

    else:
      classificacao = 'Em investigacao'

    classif_final_s.append(classificacao)

  dados['classificacao'] = classif_final_s

# colocando classificação final
  classif_final_final_s = []
  for i in dados['TPAUTOCTO']:
    if i == '1':
      classificacao_f = 'Autoctone'

    elif i == '2':
      classificacao_f = 'Importado'

    elif i == '3':
      classificacao_f = 'Importado'

    else:
      classificacao_f = ' - '

    classif_final_final_s.append(classificacao_f)

# colocando evolução
  evolucao_ = []
  for i in dados['EVOLUCAO']:
    if i == '1':
      evolucao = 'Cura'

    elif i == '2':
      evolucao = 'Óbito pelo agravo'

    elif i == '3':
      evolucao = 'Óbito por outras causas'

    elif i == '4':
      evolucao = 'Óbito em investigação'

    elif i == '9':
      evolucao = 'Ignorado'

    else:
      evolucao = ' - '

    evolucao_.append(evolucao)

  dados['classificacao_detalhada'] = classif_final_final_s
  dados['evolucao_detalhada'] = evolucao_
  dados = dados.drop( labels = ['EVOLUCAO'], axis = 1)


  dados = pd.merge(dados, municipios, on='ID_MN_RESI')
  #dados['CLASSI_FIN'].dropna(inplace=True)
  dados['endereco'] = dados['NM_LOGRADO'] +' ,'+ dados['NU_NUMERO'] + ' ,' + dados['NM_COMPLEM'] + ' ,' + dados['NM_BAIRRO']
  ordem_colunas = ['NM_PACIENT','Municipio','idade', 'DT_SIN_PRI', 'endereco',
       'DS_OBS', 'classificacao', 'classificacao_detalhada', 'DT_NOTIFIC','DT_DIGITA', 'CLASSI_FIN','TPAUTOCTO','CRITERIO', 'FEBRE', 'MIALGIA', 'CEFALEIA', 'EXANTEMA', 'VOMITO', 'NAUSEA',
       'DOR_COSTAS', 'CONJUNTVIT', 'ARTRITE', 'ARTRALGIA', 'PETEQUIA_N',
       'LEUCOPENIA', 'LACO', 'DOR_RETRO', 'HOSPITALIZ', 'evolucao_detalhada', 'NU_NOTIFIC']
  dados = dados[ordem_colunas]
  dados.columns = ['nome_do_paciente','Municipio','idade', 'inicio_sintomas', 'endereco',
       'Observacoes', 'classificacao', 'detalhe', 'data_notificacao','data_digitacao', 'CLASSI_FIN','TPAUTOCTO','CRITERIO', 'FEBRE', 'MIALGIA', 'CEFALEIA', 'EXANTEMA', 'VOMITO', 'NAUSEA',
       'DOR_COSTAS', 'CONJUNTVIT', 'ARTRITE', 'ARTRALGIA', 'PETEQUIA_N',
       'LEUCOPENIA', 'LACO', 'DOR_RETRO', 'HOSPITALIZ', 'evolucao_detalhada', 'NU_NOTIFIC']

  dados.sort_values(by='data_digitacao', ascending=False, inplace=True)

  return dados


## Buscando arquivo

In [ ]:
for i in sorted(os.listdir('/content/')):
  if i[-4:] ==".zip":
    nome_arquivo = i

In [ ]:
nome_arquivo

'1931118DBF.zip'

In [ ]:
file_name = '/content/' + nome_arquivo
with zipfile.ZipFile(file_name, 'r') as zip_ref:
      zip_ref.extractall('/content/zip')

arquivo_dbf = '/content/zip/' + sorted(os.listdir('/content/zip/'))[0]

In [ ]:
dados_dengue = limpeza_dados(arquivo_dbf)
dados_dengue

,nome_do_paciente,Municipio,idade,inicio_sintomas,endereco,Observacoes,classificacao,detalhe,data_notificacao,data_digitacao,...,CONJUNTVIT,ARTRITE,ARTRALGIA,PETEQUIA_N,LEUCOPENIA,LACO,DOR_RETRO,HOSPITALIZ,evolucao_detalhada,NU_NOTIFIC
389,LEONI TERESINHA FAGUNDES DOS SANTOS,Santa Rosa,74.0,2023-06-12,"INTERIOR ,SN , ,LAJEADO IPE","NÃO VIAJOU, NÃO TEVE DENGUE ANTERIORMENTE",Em investigacao,-,2023-06-14,2023-06-15,...,2,2,1,2,2,2,2,2,-,3073636
385,ANGELA DO AMARAL FERREIRA,Santa Rosa,31.0,2023-06-14,"RUA DO CARVOEIRO ,150 , ,AUXILIADORA",,Em investigacao,-,2023-06-15,2023-06-15,...,2,2,2,2,2,2,1,2,-,3073637
552,JOAO LUCAS DE FREITAS GARCIA OTANHA,Santa Rosa,7.0,2023-06-02,"PERIMETRAL ,235 , ,CENTRO",,Descartado,-,2023-06-05,2023-06-15,...,2,2,2,2,2,2,2,2,Cura,2934964
421,LUCAS CAMPANHONI DE BASTOS,Santa Rosa,34.0,2023-06-12,"CANDEIA ,SN , ,BELA UNIAO",,Em investigacao,-,2023-06-14,2023-06-15,...,2,2,1,2,2,2,2,,-,3073634
136,SELVINO BONNEMANN,Independência,76.0,2023-06-01,"RUA PRESIDENTE KENNEDY ,135 ,CASA ,CENTRO",,Dengue,Autoctone,2023-06-06,2023-06-14,...,2,2,1,2,2,2,1,2,Cura,2776542
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,ELIANE ANDREIA ALTISSIMO,Horizontina,43.0,2023-01-01,"RUA DA CRIANÇA ,155 , ,",,Descartado,-,2023-01-05,2023-01-06,...,2,2,2,2,2,2,2,2,Cura,3026225
919,GERALDO BERFT,Santo Cristo,52.0,2022-12-29,",SN , ,LINHA DOZE DE MAIO",,Descartado,-,2023-01-02,2023-01-04,...,1,1,1,2,2,2,1,2,Cura,0000465
99,JANICE CRISTINA KONDRA BOENO,Horizontina,48.0,2022-12-31,"RUA IPIRANGA ,34 , ,","PCTE. VIAJOU PARA ARGENTINA, EL SOBERBO, NOS 1...",Dengue,Autoctone,2023-01-04,2023-01-04,...,2,2,1,2,1,2,2,2,Cura,3026231
80,FABRICIO DESBESSEL,Horizontina,45.0,2023-01-02,"RUA IPIRANGA ,34 , ,",PACIENTE NÃO VIAJOU NOS 15 DIAS QUE ANTECEDERA...,Descartado,-,2023-01-04,2023-01-04,...,2,2,1,2,2,2,2,2,Cura,3026230


# Passando pro google sheets

In [ ]:
nome_planilha = '2023 - Casos dengue'
sh = gc.open(nome_planilha)

worksheet = sh.worksheet("dados_brutos")

dados_dengue['inicio_sintomas'] = dados_dengue['inicio_sintomas'].astype(str)
dados_dengue['data_notificacao'] = dados_dengue['data_notificacao'].astype(str)
dados_dengue['data_digitacao'] = dados_dengue['data_digitacao'].astype(str)

worksheet.update([dados_dengue.columns.values.tolist()] + dados_dengue.values.tolist())

{'spreadsheetId': '1RfzGrVP9RqpJYgLH8OKw-qr_sXZlt1R9MXHluH0SRtk',
 'updatedRange': 'dados_brutos!A1:AD1058',
 'updatedRows': 1058,
 'updatedColumns': 30,
 'updatedCells': 31740}

## Horário

In [ ]:
from datetime import date
from datetime import datetime, timedelta

today = date.today()

# dd/mm/YY
d1 = today.strftime("%m/%d/%Y")
d1

# dd/mm/YY H:M:S
dt_string = (datetime.now() - timedelta(hours=3,)).strftime("%H:%M")
lista = [dt_string, d1]
horario = pd.DataFrame(lista).T
horario.columns = ['horario', 'data']
horario

,horario,data
0,10:43,06/15/2023


In [ ]:
nome_planilha = '2023 - Casos dengue'
sh = gc.open(nome_planilha)

worksheet = sh.worksheet("horario atualizacao")

worksheet.update([horario.columns.values.tolist()] + horario.values.tolist())

{'spreadsheetId': '1RfzGrVP9RqpJYgLH8OKw-qr_sXZlt1R9MXHluH0SRtk',
 'updatedRange': "'horario atualizacao'!A1:B2",
 'updatedRows': 2,
 'updatedColumns': 2,
 'updatedCells': 4}